## 1. 디렉토리, 라이브러리 설정, 데이터 불러오기

In [ ]:
from google import colab
colab.drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
DATA_PATH = "/content/drive/Shareddrives/방세 1팀/data/"
SEED = 316

In [ ]:
import random
random.seed(SEED)
import numpy as np, pandas as pd, os
np.random.seed(SEED)
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import roc_auc_score

In [ ]:
train = pd.read_csv(f'{DATA_PATH}train.csv')

## 2. Modelling

#### 2-1. Parameter Tuning : Grid Search

  - key point는 magic 카테고리에 따라 각각 다른 모델을 만드는 것입니다. 

In [ ]:
cols = [c for c in train.columns if c not in ['id', 'target', 'magic']]

In [ ]:
# 최적화할 파라미터
grid_param = [{'reg_param': [0.1, 0.2, 0.3, 0.4, 0.5]}]
selector = VarianceThreshold(1.5)

# 각 magic 카테고리에 따라 32개의 모델을 구성 
best_params = np.zeros(32)
for i in range(32):
    train_by_m = train[train.magic == i]
    y_train = train_by_m.target
    train_by_m.reset_index(drop=True,inplace=True)
    
    # Column 별 분산에 따라 feature 선택 (분산 지수가 1.5 이하인 것들은 제외)
    data_selected = selector.fit_transform(train_by_m[cols])
    X_train = data_selected[:train_by_m.shape[0]];
    
    clf = GridSearchCV(QDA(), grid_param,cv=5)
    clf.fit(X_train, y_train)
    
    best_params[i] = clf.best_params_['reg_param']
    print("모델: " + str(i) + " / 파라미터: " + str(best_params[i]) + " / 선택된 컬럼수: " + str(len(pd.DataFrame(data_selected).columns)))

모델: 0 / 파라미터: 0.2 / 선택된 컬럼수: 45
모델: 1 / 파라미터: 0.1 / 선택된 컬럼수: 47
모델: 2 / 파라미터: 0.1 / 선택된 컬럼수: 41
모델: 3 / 파라미터: 0.1 / 선택된 컬럼수: 46
모델: 4 / 파라미터: 0.1 / 선택된 컬럼수: 33
모델: 5 / 파라미터: 0.3 / 선택된 컬럼수: 38
모델: 6 / 파라미터: 0.2 / 선택된 컬럼수: 40
모델: 7 / 파라미터: 0.1 / 선택된 컬럼수: 35
모델: 8 / 파라미터: 0.1 / 선택된 컬럼수: 46
모델: 9 / 파라미터: 0.1 / 선택된 컬럼수: 38
모델: 10 / 파라미터: 0.4 / 선택된 컬럼수: 39
모델: 11 / 파라미터: 0.4 / 선택된 컬럼수: 46
모델: 12 / 파라미터: 0.1 / 선택된 컬럼수: 35
모델: 13 / 파라미터: 0.2 / 선택된 컬럼수: 42
모델: 14 / 파라미터: 0.5 / 선택된 컬럼수: 36
모델: 15 / 파라미터: 0.1 / 선택된 컬럼수: 36
모델: 16 / 파라미터: 0.5 / 선택된 컬럼수: 39
모델: 17 / 파라미터: 0.5 / 선택된 컬럼수: 40
모델: 18 / 파라미터: 0.5 / 선택된 컬럼수: 36
모델: 19 / 파라미터: 0.1 / 선택된 컬럼수: 40
모델: 20 / 파라미터: 0.1 / 선택된 컬럼수: 42
모델: 21 / 파라미터: 0.1 / 선택된 컬럼수: 47
모델: 22 / 파라미터: 0.1 / 선택된 컬럼수: 46
모델: 23 / 파라미터: 0.3 / 선택된 컬럼수: 42
모델: 24 / 파라미터: 0.1 / 선택된 컬럼수: 43
모델: 25 / 파라미터: 0.4 / 선택된 컬럼수: 39
모델: 26 / 파라미터: 0.2 / 선택된 컬럼수: 44
모델: 27 / 파라미터: 0.4 / 선택된 컬럼수: 41
모델: 28 / 파라미터: 0.4 / 선택된 컬럼수: 46
모델: 29 / 파라미터: 0.1 / 선택된 컬럼수: 37
모델: 30 / 파라미터: 0.2 /

#### 2-2. QDA 모델 훈련 : Out-Of-Folds

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
oof_val = np.zeros(len(train))

# 각 magic 카테고리에 따라 32개의 모델을 구성
for i in range(32): 
    train_by_m = train[train.magic == i]
    idx1 = train_by_m.index
    train_by_m.reset_index(drop=True,inplace=True)
    
    # Column 별 분산에 따라 feature 선택 (분산 지수가 1.5 이하인 것들은 제외)
    train_selected = selector.fit_transform(train_by_m[cols])
    X_train = train_selected[:train_by_m.shape[0]];

    # 모델 하나 당(magic 카테고리 하나 당), 5-fold 만들어 평가 및 예측 
    skf = StratifiedKFold(n_splits=5, random_state=SEED, shuffle=True)
    for tr_idx, val_idx in skf.split(train_by_m, train_by_m.target):
        clf = QDA(reg_param = best_params[i])
        # 훈련은 X_train의 tr_idx로 진행
        clf.fit(X_train[tr_idx,:], train_by_m.loc[tr_idx].target)
        # 검증은 X_train의 val_idx로 진행
        oof_val[idx1[val_idx]] = clf.predict_proba(X_train[val_idx,:])[:,1]
      
print(f'AUC: {roc_auc_score(train.target, oof_val)}')

AUC: 0.9702484264192706
